In [1]:
!pip install quantile-forest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import time
import warnings
import pickle
import gc
import optuna
from sklearn.preprocessing import StandardScaler

from quantile_forest import RandomForestQuantileRegressor
import random
random.seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/home/a/桌面/Simulation code and data/data/time_series_15min_cleaned.csv')#res_new.csv

a = np.array(df.values[:, 2])#fujian不适用
a = np.expand_dims(a, axis=1)#fujian不适用

def split_data(data, horizon, window):
    n, m = data.shape
    X = np.zeros((n - window - horizon, window))
    Y = np.zeros((n - window - horizon, 1))

    for i in range(n - window - horizon):
        start = i
        end = start + window
        X[i, :] = data[start:end, 0]
        Y[i] = data[end + horizon - 1, 0]

    return X, Y

#fujian dataset
#def split_data(data, horizon, window):
#    n, m = data.shape
#    X = np.zeros((n - window - horizon, window))
#    Y = np.zeros((n - window - horizon, 1))
#    EX = np.zeros((n - window - horizon, 2))
#    for i in range(n - window - horizon):
#        start = i
#        end = start + window
#        X[i, :] = data[start:end, 0]
#        Y[i] = data[end + horizon - 1, 0]
#        EX[i, :] = data[end + horizon - 1, 5:6]
#    return X, Y, EX

X, Y = split_data(a, 1, 96)
n1 = X.shape[0]
#X_arg = np.hstack((X, EX))#fujian dataset
train_X, train_Y = X[:round(0.8 * n1)], Y[:round(0.8 * n1)]
val_X, val_Y = X[round(0.8 * n1):round(0.9 * n1)], Y[round(0.8 * n1):round(0.9 * n1)]
test_X, test_Y = X[round(0.9 * n1):], Y[round(0.9 * n1):]

scaler_X = StandardScaler()
x_train = scaler_X.fit_transform(train_X)
x_val = scaler_X.transform(val_X)
x_test = scaler_X.transform(test_X)

scaler_Y = StandardScaler()
y_train = scaler_Y.fit_transform(train_Y)
y_val = scaler_Y.transform(val_Y)
y_test = scaler_Y.transform(test_Y)

tau_values = [round(i, 2) for i in np.arange(0.05, 1.0, 0.05)]

In [4]:
def tilted_absolute_loss(y_true, y_pred, quantiles):
    total_loss = 0.0
    for i, tau in enumerate(quantiles):
        residual = y_true - y_pred[:, i]
        loss = np.maximum(tau * residual, (tau - 1) * residual)
        total_loss += np.sum(loss)
    
    return total_loss / (len(y_true) * len(quantiles))

In [5]:
def objective(trial):
    """
    Optuna优化目标函数
    返回验证集上的平均pinball loss（原始尺度）
    """
    # 定义超参数搜索范围
    n_estimators = trial.suggest_int('n_estimators', 10, 300, step=10)

    start_time = time.time()

    # 创建模型
    model = RandomForestQuantileRegressor(
        n_estimators=n_estimators,
        n_jobs=-1,
        random_state=42)

    # 训练模型（使用标准化后的X和Y）
    model.fit(x_train, y_train.flatten())  # 注意：quantile_forest需要一维y

    # 在验证集上预测（得到标准化尺度的预测值）
    y_pred_val_scaled = model.predict(x_val, quantiles=tau_values)

    # 将预测值反标准化到原始尺度
    y_pred_val_original = scaler_Y.inverse_transform(y_pred_val_scaled)

    # 计算平均pinball loss（原始尺度）
    current_loss = tilted_absolute_loss(val_Y, y_pred_val_original, tau_values)

    trial_time = time.time() - start_time
    trial.set_user_attr('time', trial_time)

    # 打印进度
    if trial.number % 5 == 0:
        print(f"试验 {trial.number}: n_est={n_estimators}, loss={current_loss:.6f}, 时间={trial_time:.2f}秒")

    return current_loss

In [6]:
def optimize_hyperparams(n_trials=30):
    """
    使用Optuna优化随机森林超参数
    """
    print(f"\n开始Optuna优化，共{n_trials}次试验...")
    print("搜索超参数: n_estimators")

    # 创建Optuna study
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42),
    )

    # 执行优化
    start_time = time.time()
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    total_time = time.time() - start_time

    print(f"\n{'='*60}")
    print(f"优化完成!")
    print(f"总耗时: {total_time:.3f}秒")

    # 显示最佳参数
    best_params = study.best_params
    print(f"最佳超参数:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    print(f"最佳验证损失: {study.best_value:.6f}")

    # 显示最佳试验的详细信息
    best_trial = study.best_trial
    print(f"\n最佳试验详细信息:")
    print(f"  试验编号: {best_trial.number}")
    print(f"  验证损失: {best_trial.value:.6f}")
    print(f"  运行时间: {best_trial.user_attrs.get('time', 0):.2f}秒")

    # 保存优化结果
    results = {
        'best_params': study.best_params,
        'best_loss': study.best_value,
        'best_trial_number': best_trial.number,
        'study': study,
        'total_time': total_time,
    }

    with open("optuna_qrf_results.pkl", "wb") as f:
        pickle.dump(results, f)

    print(f"\n结果已保存到 optuna_qrf_results.pkl")
    print(f"{'='*60}")

    return results

In [7]:
optuna_results = optimize_hyperparams(n_trials=30)
best_params = optuna_results['best_params']

[I 2026-01-27 13:29:48,411] A new study created in memory with name: no-name-7b0b7ed8-61ea-4129-ade2-62ddc1ccd973



开始Optuna优化，共30次试验...
搜索超参数: n_estimators


  0%|          | 0/30 [00:00<?, ?it/s]

试验 0: n_est=120, loss=2337795.256777, 时间=144.87秒
[I 2026-01-27 13:32:13,306] Trial 0 finished with value: 2337795.256777013 and parameters: {'n_estimators': 120}. Best is trial 0 with value: 2337795.256777013.
[I 2026-01-27 13:37:59,927] Trial 1 finished with value: 2336971.735345408 and parameters: {'n_estimators': 290}. Best is trial 1 with value: 2336971.735345408.
[I 2026-01-27 13:42:25,560] Trial 2 finished with value: 2337060.949182585 and parameters: {'n_estimators': 220}. Best is trial 1 with value: 2336971.735345408.
[I 2026-01-27 13:46:02,697] Trial 3 finished with value: 2337350.313981981 and parameters: {'n_estimators': 180}. Best is trial 1 with value: 2336971.735345408.
[I 2026-01-27 13:47:06,642] Trial 4 finished with value: 2338880.6453735875 and parameters: {'n_estimators': 50}. Best is trial 1 with value: 2336971.735345408.
试验 5: n_est=50, loss=2338880.645374, 时间=65.70秒
[I 2026-01-27 13:48:12,348] Trial 5 finished with value: 2338880.6453735875 and parameters: {'n_est

In [8]:
import json
best_params = {
    'n_estimators': best_params['n_estimators'],
}

with open('best_paramsqrf.json', 'w') as f:
    json.dump(best_params, f)

print("超参数已保存到 best_paramsqrf.json")

超参数已保存到 best_paramsqrf.json


In [9]:
def evaluate_on_test_set(params, save_predictions=True):
    """
    使用给定的超参数在测试集上评估模型
    """
    print(f"\n在测试集上评估...")
    print(f"超参数: {params}")

    start_time = time.time()

    # 创建模型
    model = RandomForestQuantileRegressor(
        n_estimators=params['n_estimators'],
        n_jobs=-1,
        random_state=42
    )

    # 训练模型（使用标准化后的训练集X和Y）
    print("训练模型...")
    model.fit(x_train, y_train.flatten())

    eval_time1 = time.time() - start_time
    # 在测试集上预测（得到标准化尺度的预测值）
    print("在测试集上预测...")
    start_time2 = time.time()
    y_pred_test_scaled = model.predict(x_test, quantiles=tau_values)

    # 将预测值反标准化到原始尺度
    y_pred_test_original = scaler_Y.inverse_transform(y_pred_test_scaled)

    test_loss = tilted_absolute_loss(test_Y, y_pred_test_original, tau_values)

    eval_time2 = time.time() - start_time2

    print(f"\n{'='*60}")
    print(f"测试集评估结果:")
    print(f"测试集平均pinball loss: {test_loss:.6f}")
    print(f"训练评估总耗时: {eval_time1:.3f}秒")
    print(f"测试评估总耗时: {eval_time2:.3f}秒")

    return test_loss, y_pred_test_original, model

In [10]:
test_loss, test_predictions_matrix, final_model = evaluate_on_test_set(best_params)


在测试集上评估...
超参数: {'n_estimators': 270}
训练模型...
在测试集上预测...

测试集评估结果:
测试集平均pinball loss: 2401483.753809
训练评估总耗时: 321.665秒
测试评估总耗时: 2.028秒


In [11]:
pred = pd.DataFrame(test_predictions_matrix)
pred.to_excel('predqrf.xlsx', index=True)

In [12]:
best_params = optuna_results['best_params']

# 5. 重新构建并训练模型
print("\n使用最佳参数重新训练模型...")
model = RandomForestQuantileRegressor(
    n_estimators=best_params['n_estimators'],
    n_jobs=-1,
    random_state=42
)

model.fit(x_train, y_train.flatten())

# 6. 在测试集上进行评估
start_time2 = time.time()
print("在测试集上评估...")
y_pred_test_scaled = model.predict(x_test, quantiles=tau_values)
y_pred_test_original = scaler_Y.inverse_transform(y_pred_test_scaled)

test_loss = tilted_absolute_loss(test_Y, y_pred_test_original, tau_values)
print(f"测试集平均pinball loss: {test_loss:.6f}")
eval_time2 = time.time() - start_time2
print(f"测试评估总耗时: {eval_time2:.3f}秒")


使用最佳参数重新训练模型...
在测试集上评估...
测试集平均pinball loss: 2401483.753809
测试评估总耗时: 1.826秒
